<a href="https://colab.research.google.com/github/Djaxis/MY-Python-Evolution/blob/main/Tranches_d'age_touch%C3%A9es_par_le_Covid_Brice_De_Campos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###ATTENTION LE FICHIER DE DATA EST MIS A JOUR CHAQUE ANNEES###
## **ANALYSE DES DONNEES  hospitalières à partir DU 18 MARS 2020 19h06 : Hospitalisation Réanimation Décès**

Source "Données de santé public france" : page 2
https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/

In [10]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.offline.offline import plot
from plotly.subplots import make_subplots
from datetime import datetime

In [11]:
## Récupération de données brut
df = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/08c18e08-6780-452d-9b8c-ae244ad529b3', sep=";")
df.head()

,reg,cl_age90,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc
0,1,0,2020-03-18,0,0,NaN,NaN,NaN,0,0
1,1,9,2020-03-18,0,0,NaN,NaN,NaN,0,0
2,1,19,2020-03-18,0,0,NaN,NaN,NaN,0,0
3,1,29,2020-03-18,0,0,NaN,NaN,NaN,0,0
4,1,39,2020-03-18,0,0,NaN,NaN,NaN,0,0


In [12]:
## Ici je vais chercher dans le fichier les âges proposés, ce qui me permettra de faire des tranches d'âge.
df['cl_age90'].unique() ## 0 = tout le monde, 9 = des 0à 10 ans, 19 = de 10 à 20 ans.

array([ 0,  9, 19, 29, 39, 49, 59, 69, 79, 89, 90])

In [13]:
## Fonction qui organise par tranches d'âge
def reduce(age):
    if age in [9, 19, 29]:
        return '-30 ans'
    if age in [39, 49]:
        return '30-50 ans'
    if age in [59, 69]:
        return '50-70 ans'
    if age in [69, 79]:
        return '60-80 ans'
    if age in [89, 90]:
        return '+80 ans'

## Création de la collonne age qui est = la colonne 'cl_age90' surlaquelle j'applique la fonction reduce      
df['age'] = df['cl_age90'].apply(reduce)
df


,reg,cl_age90,jour,hosp,rea,HospConv,SSR_USLD,autres,rad,dc,age
0,1,0,2020-03-18,0,0,NaN,NaN,NaN,0,0,None
1,1,9,2020-03-18,0,0,NaN,NaN,NaN,0,0,-30 ans
2,1,19,2020-03-18,0,0,NaN,NaN,NaN,0,0,-30 ans
3,1,29,2020-03-18,0,0,NaN,NaN,NaN,0,0,-30 ans
4,1,39,2020-03-18,0,0,NaN,NaN,NaN,0,0,30-50 ans
...,...,...,...,...,...,...,...,...,...,...,...
212053,94,59,2023-02-21,2,0,1.0,1.0,0.0,385,14,50-70 ans
212054,94,69,2023-02-21,12,0,5.0,5.0,2.0,534,48,50-70 ans
212055,94,79,2023-02-21,29,0,13.0,15.0,1.0,798,128,60-80 ans
212056,94,89,2023-02-21,39,1,25.0,13.0,0.0,861,199,+80 ans


In [14]:
## Supprimer la colonne cl_age90 et grouper age et jour en index : je vais cherhcer pour chaque jour la sum des + de 80 par exemple etc...
dataTranchesAges = df.drop('cl_age90', axis=1).groupby(['age', 'jour']).sum()
dataTranchesAges

reg  hosp  rea  HospConv  SSR_USLD  autres     rad  \
age       jour                                                              
+80 ans   2020-03-18  1418   826   94       0.0       0.0     0.0      76   
          2020-03-19  1418  1314  125       0.0       0.0     0.0     144   
          2020-03-20  1418  1498  131       0.0       0.0     0.0     192   
          2020-03-21  1418  1764  148       0.0       0.0     0.0     237   
          2020-03-22  1418  2022  161       0.0       0.0     0.0     281   
...                    ...   ...  ...       ...       ...     ...     ...   
60-80 ans 2023-02-17   709  3075  232    1629.0    1169.0    45.0  163118   
          2023-02-18   709  3057  228    1616.0    1168.0    45.0  163154   
          2023-02-19   709  3052  228    1615.0    1164.0    45.0  163161   
          2023-02-20   709  3085  221    1653.0    1167.0    44.0  163216   
          2023-02-21   709  3080  222    1655.0    1160.0    43.0  163313   

                         dc  
age       jour               
+80 ans   2020-03-18    146  
          2020-03-19    236  
          2020-03-20    283  
          2020-03-21    348  
          2020-03-22    413  
...                     ...  
60-80 ans 2023-02-17  30604  
          2023-02-18  30609  
          2023-02-19  30610  
          2023-02-20  30616  
          2023-02-21  30621  

[5355 rows x 8 columns]

In [15]:
## je crée groups pour mettre des couleurs par tranches d'age à mon futur grafique.
groups = [('-30 ans', 'rgb(255, 0, 127)'),
          ('30-50 ans', 'rgb(121, 28, 248)'), 
          ('50-70 ans', 'rgb(0, 255, 127)'), 
          ('60-80 ans', 'rgb(246,207,113)'), 
          ('+80 ans', 'rgb(102,197,204)')]

## Création du graphique
fig = go.Figure()

for i, c in groups:
        x = dataTranchesAges.loc[i] 
        fig.add_trace(go.Bar(name=i, x=x.index, y=x['hosp'], marker_color=c)) ## en axis l'index donc age/jour et en y la colonne hosp.
fig.update_layout(barmode='stack', title="Nombre d'hospitalisations/jour (Mettre la souris sur le graph pour voir les jours, sélèctionner un groupe de bars pour zoumer sur une période)")

plot(fig, include_plotlyjs=False, include_mathjax=False, output_type='div')
fig.show()

# **Analyse des hospitalisations par tranches d'âge : Attention il ne sagit pas de nouvelles hospitalisations, mais de l'état du nombre d'hospitalisations sur le jour même. Nous n'avons pas les données de co-morbidités et antécédants des patients sur ce graph

En cliquand sur les tranches d'age on peut les activer ou pas. 


In [16]:
fig = go.Figure()

for i, c in groups:
        x = dataTranchesAges.loc[i]
        fig.add_trace(go.Bar(name=i, x=x.index, y=x['rea'], marker_color=c))
fig.update_layout(barmode='stack', title="Nombre de réaminations/jour (Mettre la souris sur le graph pour voir les jours, sélèctionner un groupe de bars pour zoumer sur une période)")

plot(fig, include_plotlyjs=False, include_mathjax=False, output_type='div')
fig.show()

## **Analyse des réanimation par tranches d'âgeAttention il ne sagit pas de nouvelles réanimation, mais de l'état du nombre de réa sur le jour même. Nous n'avons pas les données de co-morbidités et antécédants des patients sur ce graph ##
En cliquand sur les tranches d'age on peut les activer ou pas. 
On remarque 

# ▶ **Les plus de 50 ans sont le plus concerné par la réanimation**

In [17]:
fig = go.Figure()

for i, c in groups:
        x = dataTranchesAges.loc[i]
        fig.add_trace(go.Bar(name=i, x=x.index, y=(x['dc'] - x['dc'].shift(1)).clip(lower=0), marker_color=c))
fig.update_layout(barmode='stack', title="Nombre de décès/jour (Mettre la souris sur le graph pour voir les jours, sélèctionner un groupe de bars pour zoumer sur une période)")

plot(fig, include_plotlyjs=False, include_mathjax=False, output_type='div')
fig.show()

# **Analyse des décès par tranches d'âge(Attention il ne sagit pas de nouvelles réanimation, mais de l'état du nombre de décès sur le jour même. Nous n'avons pas les données de co-morbidités et antécédants des patients sur ce graph)**
En cliquand sur les tranches d'age on peut les activer ou pas. 
On remarque 

# ▶ **Les plus de 50 ans sont le plus concerné par les décès**

# ▶ **Les plus de 80 ans ont le tx de mortalité le plus fort du groupe 50 80 ans**

# ▶ **C'est une épidémie qui tue essentiellement les + de 80 ans**

In [18]:
fig = go.Figure()

for i, c in groups:
        x = dataTranchesAges.loc[i]
        fig.add_trace(go.Bar(name=i, x=x.index, y=x['dc'], marker_color=c))
fig.update_layout(barmode='stack', title="Nombre de décès cumulés")


plot(fig, include_plotlyjs=False, include_mathjax=False, output_type='div')

fig.show()

# **Analyse des décès cumulés par tranches d'âge : du 18 mars 2020 au 21 Décembre 2021** (Attention nous n'avons pas les données de co-morbidités et antécédants des patients sur ce graph)
En cliquand sur les tranches d'age on peut les activer ou pas. 
On remarque 
# ▶ **120 personnes de moins de 30 ans sont mortes sur 2 ans**

# ▶ **Les plus de 50 ans sont le plus concerné par les décès**

# ▶ **Les plus de 80 ans ont le tx de mortalité le plus fort du groupe 50 80 ans**

# ▶ **C'est une épidémie qui tue essentiellement les + de 80 ans**